<a href="https://colab.research.google.com/github/fabiobasson/my-repo/blob/main/sonar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import numpy
import pandas
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = pandas.read_csv("https://raw.githubusercontent.com/fabiobasson/my-repo/main/datasets/sonar.csv", header=1)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
y = dataset[:,60]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X,encoded_y, test_size=0.2, random_state=42)

X_val, y_val = X_train[50:], y_train[50:]

In [102]:
# baseline model
def create_baseline():
# create model
  model = Sequential()
  model.add(Dense(60, input_dim=60, activation= 'relu' ))
  model.add(Dense(30, input_dim=60, activation= 'relu' ))
  model.add(Dense(1, activation= 'sigmoid' ))
  # Compile model
  model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
  return model

In [98]:
  model = Sequential()
  model.add(Dense(60, input_dim=60, activation= 'relu'))
  model.add(Dense(30, input_dim=60, activation= 'relu'))
  model.add(Dense(1, activation= 'sigmoid'))

#Compile model
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])

In [100]:
history = model.fit(X_train,y_train,
                               epochs=100,
                               verbose=0,
                               validation_data=(X_val,y_val))

In [101]:
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 78.57%


In [105]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)

estimators = []
estimators.append(( 'standardize' , StandardScaler()))
estimators.append(( 'mlp' , KerasClassifier(build_fn=create_baseline, nb_epoch=100,batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(pipeline, X_train, y_train, cv=kfold)

print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 76.76% (9.84%)
